**import des librairies**

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

import wandb

**paramètres de configuration**

In [ ]:
DATA_DIR = './data/'
KAGGLE_DATASET_NAME = 'xhlulu/siim-covid19-resized-to-512px-jpg'
TRAIN_PATH = DATA_DIR + 'train/'
TEST_PATH = DATA_DIR + 'test/'
IMG_SIZE = 512
NB_IMAGES_VISUALISATION = 42

**téléchargement automatique des données**

In [ ]:
if not os.path.isdir(DATA_DIR):
    !kaggle datasets download {KAGGLE_DATASET_NAME} --path {DATA_DIR} --unzip --quiet
    !kaggle competitions download -c "siim-covid19-detection" --file "sample_submission.csv" --path {DATA_DIR} --quiet
    !kaggle competitions download -c "siim-covid19-detection" --file "train_image_level.csv" --path {DATA_DIR} --quiet
    !kaggle competitions download -c "siim-covid19-detection" --file "train_study_level.csv" --path {DATA_DIR} --quiet
    
    file_to_extract = DATA_DIR+"/train_image_level.csv.zip"
    with zipfile.ZipFile(file_to_extract) as file:
        file.extract("train_image_level.csv", DATA_DIR)
    os.remove(file_to_extract)

**import des méta données**